In [1]:
import sys
print(sys.path)
sys.path.append('G:\michru')

['g:\\michru\\jupyter', 'c:\\Users\\nyanm\\.vscode\\extensions\\ms-python.python-2020.4.76186\\pythonFiles', 'c:\\Users\\nyanm\\.vscode\\extensions\\ms-python.python-2020.4.76186\\pythonFiles\\lib\\python', 'g:\\michru\\.venv\\Scripts\\python36.zip', 'C:\\Python36\\DLLs', 'C:\\Python36\\lib', 'C:\\Python36', 'g:\\michru\\.venv', '', 'g:\\michru\\.venv\\lib\\site-packages', 'g:\\michru\\.venv\\lib\\site-packages\\win32', 'g:\\michru\\.venv\\lib\\site-packages\\win32\\lib', 'g:\\michru\\.venv\\lib\\site-packages\\Pythonwin', 'g:\\michru\\.venv\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\nyanm\\.ipython']


In [44]:
from utils.sql_ops import LevelOps
import pandas as pd
import numpy as np
import sqlite3

In [3]:
class SqlOps():
    def __init__(self):
        self.conn = sqlite3.connect('starlightRe.db')
        self.c = self.conn.cursor()
    
    def create(self, cols, types, primary):
        ct = []
        for c, t in zip(cols, types):
            ct.append("{} {}".format(c, t))
        sqls = 'CREATE TABLE IF NOT EXISTS {} ({}, PRIMARY KEY ({}))'.format(self.table, ','.join(ct), primary)
        try: 
            self.c.execute(sqls)
        except Exception as e:
            print("error: ", e)
            return 0            
        return 1
    
    def insert(self, data):
        sqls = 'INSERT OR IGNORE INTO ' + self.table + ' VALUES ({}?)'.format('?,'*(len(data[0]) - 1))
        print(sqls)
        try:
            self.c.executemany(sqls, iter(data))
            self.conn.commit()
        except Exception as e:
            print("error: ", e)
            self.conn.rollback()
    
    def find_all(self):
        sqls = 'SELECT * FROM {}'.format(self.table)
        results = self.c.execute(sqls)
        return results
    
    def update(self):
        pass
    
    def delete(self):
        pass

    def close(self):
        self.c.close()
        self.conn.close()

In [4]:
class DressOps(SqlOps):
    def __init__(self):
        super(DressOps, self).__init__()
        self.table = 'dress'
    
    def find_all(self):
        sqls = 'SELECT * FROM {}'.format(self.table)
        df = pd.read_sql_query(sqls, self.conn)
        return df

In [29]:
Dress = DressOps()
ddata = Dress.find_all()

In [21]:

ddata.head()

,id,name,chara_id,cost,base_agi,base_rarity,growth_board3_id,base_pdef,delta_agi,dex,...,hit,base_mdef,growth_board2_id,growth_board9_id,command_skill3_id,published_at,command_unique_skill_id,dress_type,base_hp,description
0,1010001,聖翔音楽学園,101,6,94,2,11101301,77,1560,5,...,0,75,11101201,11101901,11,1540105200,12,0,1336,みんなを守る盾役。通常ダメージを防ぐバリアを張ることで長く舞台に残ることができる。
1,1010002,太陽の国の騎士,101,9,129,3,11105301,58,2130,5,...,0,90,11105201,11105901,16,1540105200,17,0,1128,バランス良いステータスを持つため、様々なステージで活躍できる。敵役単体に大ダメージを与えるク...
2,1010003,銃士ダルタニャン,101,12,165,4,11106301,74,2730,5,...,0,78,11106201,11106901,21,1540105200,22,0,1007,高いACTパワーを持ち、ダメージを与えるのが得意。敵役の防御力を下げつつ、自ら全体攻撃のクラ...
3,1020001,聖翔音楽学園,102,6,120,2,11101301,53,1990,5,...,0,51,11101201,11101901,26,1540105200,27,0,794,高いACTパワーを持ち、ダメージを与えるのが得意。敵役のACTパワーを低下させるクライマック...
4,1020002,黒獅子の国の騎士,102,9,139,3,11105301,77,2290,5,...,0,52,11105201,11105901,31,1540105200,32,0,900,高いACTパワーを持ち、ダメージを与えるのが得意。クライマックスACTで敵役にダメージを与え...


In [26]:
ddata[ddata['id'] == '1010009']

,id,name,chara_id,cost,base_agi,base_rarity,growth_board3_id,base_pdef,delta_agi,dex,...,hit,base_mdef,growth_board2_id,growth_board9_id,command_skill3_id,published_at,command_unique_skill_id,dress_type,base_hp,description
164,1010009,怪傑ゾロ,101,12,178,4,13204301,52,2950,5,...,0,76,13204201,13204901,1648,1581577200,1649,0,885,敵役に対しては刻印付与や継続プラス効果解除を行い、味方全体には、クライマックスACTで被ダメ...


数值型列：base_agi,base_pdef,delta_agi,dex,delta_pdef, base_atk, delta_atk, delta_hp, delta_mdef, hit(不归一化), base_mdef, base_hp

category型： growth_board3_id, growth_board7_id, growth_board1_id, party_skill_type, growth_board6_id, attribute_id, growth_board4_id, auto_skill2_id, party_skill1_id, auto_skill2_type, auto_skill1_id, command_skill1_id, auto_skill1_type, attack_type, growth_board5_id, command_skill2_id, auto_skill3_type, auto_skill3_id, growth_board8_id, growth_board2_id, growth_board9_id, command_skill3_id, command_unique_skill_id, dress_type, 

文字类型：description

In [7]:
class LevelOps(SqlOps):
    def __init__(self):
        super(LevelOps, self).__init__()
        self.table = 'Level'
        self.clos = ''
    
    def find_all(self):
        sqls = 'SELECT * FROM {}'.format(self.table)
        df = pd.read_sql_query(sqls, self.conn)
        return df
    
    def update(self, data):
        sqls = 'REPLACE INTO {} VALUES({}?)'.format(self.table, '?,'*(len(data[0]) - 1))
        print(sqls)
        try:
            self.c.executemany(sqls, iter(data))
            self.conn.commit()
        except Exception as e:
            print("error: ", e)
            self.conn.rollback()

In [8]:

Level = LevelOps()
ldata = Level.find_all()

In [9]:
ldata = ldata[['id', 'level']]
ldata = ldata.set_index('id')

In [10]:
ldata.head()

,level
id,
1010009,2
1020005,2
1020007,3
1020008,2
1020009,5


In [30]:
ddata = ddata[ddata['base_rarity'] == 4]
ddata = ddata[['id', 'base_agi','base_pdef','delta_agi','dex','delta_pdef', 'base_atk', 'delta_atk', 'delta_hp', 'delta_mdef', 'hit', 'base_mdef', 'base_hp', 'growth_board3_id', 'growth_board7_id', 'growth_board1_id', 'party_skill_type', 'growth_board6_id', 'attribute_id', 'growth_board4_id', 'auto_skill2_id', 'party_skill_id', 'auto_skill2_type', 'auto_skill1_id', 'command_skill1_id', 'auto_skill1_type', 'attack_type', 'growth_board5_id', 'command_skill2_id', 'auto_skill3_type', 'auto_skill3_id', 'growth_board8_id', 'growth_board2_id', 'growth_board9_id', 'command_skill3_id', 'command_unique_skill_id', 'dress_type']]
ddata['id'] = ddata['id'].astype(int)
# ddata = ddata.set_index('id')

In [33]:
ddata = ddata.set_index('id')

In [34]:
ddata.head()

,base_agi,base_pdef,delta_agi,dex,delta_pdef,base_atk,delta_atk,delta_hp,delta_mdef,hit,...,growth_board5_id,command_skill2_id,auto_skill3_type,auto_skill3_id,growth_board8_id,growth_board2_id,growth_board9_id,command_skill3_id,command_unique_skill_id,dress_type
id,,,,,,,,,,,,,,,,,,,,,
1010003,165,74,2730,5,1220,156,2580,33180,1290,0,...,11106501,20,2,20024,11106801,11106201,11106901,21,22,0
1020003,160,76,2640,5,1260,163,2700,38100,1250,0,...,11106501,35,2,20024,11106801,11106201,11106901,36,37,0
1030003,187,73,3090,5,1210,140,2310,37630,1260,0,...,12104501,50,1,10076,12104801,12104201,12104901,51,52,0
1050002,134,134,2210,5,2210,93,1540,59910,1580,0,...,11105501,75,2,20024,11105801,11105201,11105901,76,77,0
1060003,170,63,2810,5,1040,176,2910,26340,1010,0,...,13202501,90,1,10074,13202801,13202201,13202901,91,92,0


In [36]:
train = ddata.join(ldata)

In [37]:
train.head()

,base_agi,base_pdef,delta_agi,dex,delta_pdef,base_atk,delta_atk,delta_hp,delta_mdef,hit,...,command_skill2_id,auto_skill3_type,auto_skill3_id,growth_board8_id,growth_board2_id,growth_board9_id,command_skill3_id,command_unique_skill_id,dress_type,level
id,,,,,,,,,,,,,,,,,,,,,
1010003,165,74,2730,5,1220,156,2580,33180,1290,0,...,20,2,20024,11106801,11106201,11106901,21,22,0,NaN
1020003,160,76,2640,5,1260,163,2700,38100,1250,0,...,35,2,20024,11106801,11106201,11106901,36,37,0,NaN
1030003,187,73,3090,5,1210,140,2310,37630,1260,0,...,50,1,10076,12104801,12104201,12104901,51,52,0,NaN
1050002,134,134,2210,5,2210,93,1540,59910,1580,0,...,75,2,20024,11105801,11105201,11105901,76,77,0,NaN
1060003,170,63,2810,5,1040,176,2910,26340,1010,0,...,90,1,10074,13202801,13202201,13202901,91,92,0,NaN


In [36]:
len(train)

142

In [39]:
train.columns

Index(['base_agi', 'base_pdef', 'delta_agi', 'dex', 'delta_pdef', 'base_atk',
       'delta_atk', 'delta_hp', 'delta_mdef', 'hit', 'base_mdef', 'base_hp',
       'growth_board3_id', 'growth_board7_id', 'growth_board1_id',
       'party_skill_type', 'growth_board6_id', 'attribute_id',
       'growth_board4_id', 'auto_skill2_id', 'party_skill_id',
       'auto_skill2_type', 'auto_skill1_id', 'command_skill1_id',
       'auto_skill1_type', 'attack_type', 'growth_board5_id',
       'command_skill2_id', 'auto_skill3_type', 'auto_skill3_id',
       'growth_board8_id', 'growth_board2_id', 'growth_board9_id',
       'command_skill3_id', 'command_unique_skill_id', 'dress_type', 'level'],
      dtype='object')

In [41]:
train[['base_agi', 'base_pdef', 'delta_agi', 'delta_pdef','base_atk','delta_atk', 'delta_hp', 'delta_mdef', 'base_mdef', 'base_hp']] = train[['base_agi', 'base_pdef', 'delta_agi', 'delta_pdef','base_atk','delta_atk', 'delta_hp', 'delta_mdef', 'base_mdef', 'base_hp']].apply(lambda x: (x - np.min(x))/(np.max(x) - np.min(x)))

In [42]:
train.head()

,base_agi,base_pdef,delta_agi,dex,delta_pdef,base_atk,delta_atk,delta_hp,delta_mdef,hit,...,command_skill2_id,auto_skill3_type,auto_skill3_id,growth_board8_id,growth_board2_id,growth_board9_id,command_skill3_id,command_unique_skill_id,dress_type,level
id,,,,,,,,,,,,,,,,,,,,,
1010003,0.616667,0.265487,0.564815,5,0.262032,0.356757,0.459916,0.229389,0.327684,0,...,20,2,20024,11106801,11106201,11106901,21,22,0,NaN
1020003,0.533333,0.283186,0.481481,5,0.283422,0.394595,0.510549,0.367358,0.305085,0,...,35,2,20024,11106801,11106201,11106901,36,37,0,NaN
1030003,0.983333,0.256637,0.898148,5,0.256684,0.270270,0.345992,0.354178,0.310734,0,...,50,1,10076,12104801,12104201,12104901,51,52,0,NaN
1050002,0.100000,0.796460,0.083333,5,0.791444,0.016216,0.021097,0.978968,0.491525,0,...,75,2,20024,11105801,11105201,11105901,76,77,0,NaN
1060003,0.700000,0.168142,0.638889,5,0.165775,0.464865,0.599156,0.037577,0.169492,0,...,90,1,10074,13202801,13202201,13202901,91,92,0,NaN


In [43]:
pd.get_dummies(train[['growth_board3_id', 'growth_board7_id', 'growth_board1_id',
       'party_skill_type', 'growth_board6_id', 'attribute_id',
       'growth_board4_id', 'auto_skill2_id', 'party_skill_id',
       'auto_skill2_type', 'auto_skill1_id', 'command_skill1_id',
       'auto_skill1_type', 'attack_type', 'growth_board5_id',
       'command_skill2_id', 'auto_skill3_type', 'auto_skill3_id',
       'growth_board8_id', 'growth_board2_id', 'growth_board9_id',
       'command_skill3_id', 'command_unique_skill_id', 'dress_type']])

,party_skill_type,attribute_id,auto_skill2_type,auto_skill1_type,attack_type,auto_skill3_type,dress_type,growth_board3_id_11101301,growth_board3_id_11102301,growth_board3_id_11103301,...,command_unique_skill_id_869,command_unique_skill_id_874,command_unique_skill_id_892,command_unique_skill_id_897,command_unique_skill_id_902,command_unique_skill_id_92,command_unique_skill_id_932,command_unique_skill_id_953,command_unique_skill_id_963,command_unique_skill_id_990
id,,,,,,,,,,,,,,,,,,,,,
1010003,5,6,2,1,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1020003,5,6,2,1,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1030003,5,4,2,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1050002,5,5,1,1,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1060003,5,2,2,1,2,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1080008,5,5,2,2,2,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2040008,5,1,2,2,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2010007,5,2,1,2,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
